<a href="https://colab.research.google.com/github/lucasbarrosrocha/rec-info/blob/master/recinfo_lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### Recuperação da Informação
#### Professor: Leandro Balby
#### Aluno: Lucas Barros Rocha - 115111579



In [0]:
#bibliotecas
import pandas as pd
import numpy as np
import nltk
import re
import collections
import heapq
import time
import bisect
import random
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
result = pd.read_csv('https://raw.githubusercontent.com/lucasbarrosrocha/rec-info/master/ri_lab_06/results.csv') 
                                  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indexes = {}
n = 0

for text in result.text:
  words = [word for word in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in indexes.keys():
      indexes[t] = []
    indexes[t].append(n)
    
for elem in indexes.items():
  d = dict(collections.Counter(elem[1]))
  indexes[elem[0]] = list(d.items())

In [0]:
queries = ["bolsonaro", "educação", "ministério", "brasil", "economia"]

### 1. Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que?

In [0]:
def n_docs_word(word, ranking):
  return len(ranking[word])

def n_docs_both(a, b, ranking):
  docs_a = [doc for doc,score in ranking[a]]
  docs_b = [doc for doc,score in ranking[b]]
  n = 0
  
  for doc in docs_a:
    if doc in docs_b:
      n += 1
      
  return n

In [0]:
def calc_mim(a, b, ranking):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  mim = n_ab / quo if quo != 0 else 0
  return (mim,b)
  
def calc_emim(a, b, ranking, n):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  exp = n * (n_ab / quo) if quo != 0 else 0
  emim = n_ab * np.log10(exp) if exp != 0 else 0
  return (emim, b)
  
def calc_x(a, b, ranking, n):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  x = (n_ab - (1 / n) * n_a * n_b) ** 2 / quo if quo != 0 else 0
  return (x,b)
  
def calc_dice(a, b, ranking):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  dice = n_ab / (n_a + n_b)
  return (dice,b)

In [0]:
def create_metrics_table(query, ranking, n):
  top_10_table = pd.DataFrame()
  MIM = []
  EMIM = []
  X = []
  Dice = []
  
  for word in ranking:
    if word != query:
      bisect.insort(MIM, calc_mim(query, word, ranking))
      bisect.insort(EMIM,calc_emim(query, word, ranking, n))
      bisect.insort(X,calc_x(query, word, ranking, n))
      bisect.insort(Dice,calc_dice(query, word, ranking))

  MIM.reverse()
  EMIM.reverse()
  X.reverse()
  Dice.reverse()
  
  top_10_table['MIM'] = [word[1] for word in MIM[:10]]
  top_10_table['EMIM'] = [word[1] for word in EMIM[:10]]
  top_10_table['X²'] = [word[1] for word in X[:10]]
  top_10_table['Dice'] = [word[1] for word in Dice[:10]]
  
  top_10_table.index += 1
  
  return top_10_table

In [0]:
n = result.text.count()
table_1 = create_metrics_table(queries[0], indexes, n)
table_2 = create_metrics_table(queries[1], indexes, n)
table_3 = create_metrics_table(queries[2], indexes, n)
table_4 = create_metrics_table(queries[3], indexes, n)
table_5 = create_metrics_table(queries[4], indexes, n)

####1.1 Top-10 palavras associadas a 'Bolsonaro' 

In [0]:
table_1

,MIM,EMIM,X²,Dice
1,ótimas,presidente,jair,presidente
2,óbvios,brasil,psl,brasil
3,óbvias,governo,previdência,governo
4,índole,brasileiro,reforma,brasileiro
5,ético,ministro,planalto,ministro
6,ânus,reforma,brasileiro,político
7,ânimos,político,brasil,brasileira
8,ângulos,psl,deputado,ainda
9,ângelo,jair,presidente,paulo
10,ávidos,previdência,guedes,federal


#### 1.2 Top-10 palavras associadas a 'Educação'

In [0]:
table_2

,MIM,EMIM,X²,Dice
1,óbvios,alunos,alunos,alunos
2,óbvias,escolas,vélez,escolas
3,óbvia,social,escolas,social
4,étnicos,escola,mec,mudanças
5,étnico,discurso,aulas,escola
6,átomo,governo,aula,discurso
7,árbitros,mudanças,aluno,comissão
8,zombava,comissão,pasta,necessidade
9,zeina,vélez,estudantes,jovens
10,zeca,pasta,educacional,pasta


#### 1.3 Top-10 palavras associadas a 'Ministério'

In [0]:
table_3

,MIM,EMIM,X²,Dice
1,úteis,federal,pasta,federal
2,óbvios,ministro,federal,ministro
3,íntegra,público,feita,público
4,ídolos,militares,valores,militares
5,étnico,valores,vieira,valores
6,ético,feita,promovido,governo
7,ânus,psl,lamenta,disse
8,ângulos,governo,psl,rio
9,ângelo,pasta,cidadania,justiça
10,áudios,disse,ministro,chefe


#### 1.4 Top-10 palavras associadas a 'Brasil'

In [0]:
table_4

,MIM,EMIM,X²,Dice
1,útero,bolsonaro,brasileiros,bolsonaro
2,úteis,brasileiros,bolsonaro,sobre
3,ótimas,brasileira,brasileira,ainda
4,órgãos,sociais,paulo,governo
5,óbvios,paulo,sociais,ser
6,óbvias,brasileiro,brasileiro,presidente
7,óbvia,governo,brasileiras,anos
8,índole,federal,etc,país
9,índios,ainda,federal,sociais
10,índices,aqui,órgãos,outros


#### 1.5 Top-10 palavras associadas a 'Economia'

In [0]:
table_5

,MIM,EMIM,X²,Dice
1,úteis,reforma,previdência,reforma
2,úlceras,previdência,reforma,previdência
3,óbvios,mercado,guedes,mercado
4,óbitos,econ,mercado,econ
5,índices,guedes,maia,governo
6,êxito,governo,mercados,menos
7,étnicos,mica,reformas,mica
8,ânus,menos,aposentadorias,política
9,ânsias,reformas,incerteza,presidente
10,ávidos,política,economista,ministro


### 2. De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a precisão dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.

In [0]:
def term_a_time(query, indexes, k):    
    a = {} # Accumulator HashTable
    query_indexes = [] # Array
    rank = [] # HeapQueue, implementation of PriorityQueue
    
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    
    for index in query_indexes:
      for i in index:
          doc = i[0]
          if doc not in a.keys():
              a[doc] = 0
          a[doc] += i[1]
    
    for (doc, score) in a.items():
      heapq.heappush(rank, (score, doc))
        
    return heapq.nlargest(k, rank)

In [0]:
def top_docs(query, k):
  
  top_docs = [doc for score,doc in term_a_time(query, indexes, k)]
  top_docs_ranking = {} 
  
  for word in indexes:
    top_docs_ranking[word] = []
    for doc_score in indexes[word]:
      if doc_score[0] in top_docs:
        top_docs_ranking[word].append(doc_score)
        
  return top_docs_ranking

#### Top-3 Documentos

Expansão de consulta para os top-3 documentos das consultas selecionadas inicialmente.

In [0]:
k = 3
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [0]:
table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

##### Top-10 palavras associadas aos Top-3 Documentos de 'Bolsonaro'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,presidente,vez
2,brasil,três
3,governo,trata
4,brasileiro,trabalho
5,ministro,todas
6,político,sobre
7,brasileira,sob
8,ainda,site
9,paulo,sendo
10,federal,república


##### Top-10 palavras associadas aos Top-3 Documentos de 'Educação'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,alunos,todo
2,escolas,social
3,social,ser
4,mudanças,qualidade
5,escola,proposta
6,discurso,projeto
7,comissão,parece
8,necessidade,onde
9,jovens,fazendo
10,pasta,faz


##### Top-10 palavras associadas aos Top-3 Documentos de 'Ministério'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,federal,vez
2,ministro,ter
3,público,ser
4,militares,reais
5,valores,pública
6,governo,projeto
7,disse,primeiro
8,rio,política
9,justiça,parte
10,chefe,ministro


##### Top-10 palavras associadas aos Top-3 Documentos de 'Brasil'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,vão
2,sobre,voto
3,ainda,violência
4,governo,vida
5,ser,vezes
6,presidente,vez
7,anos,vergonha
8,país,verdade
9,sociais,vai
10,outros,têm


##### Top-10 palavras associadas aos Top-3 Documentos de 'Economia'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,reforma,sobre
2,previdência,setor
3,mercado,ser
4,econ,segundo
5,governo,resultado
6,menos,relação
7,mica,quase
8,política,principalmente
9,presidente,presidente
10,ministro,pouco


#### Top-5 Documentos

Expansão de consulta para os top-5 documentos das consultas selecionadas inicialmente.

In [0]:
k = 5
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [0]:
table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

##### Top-10 palavras associadas aos Top-5 Documentos de 'Bolsonaro'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,presidente,sobre
2,brasil,quase
3,governo,presidente
4,brasileiro,porém
5,ministro,pessoas
6,político,outro
7,brasileira,militar
8,ainda,fazer
9,paulo,envolvendo
10,federal,conta


##### Top-10 palavras associadas aos Top-5 Documentos de 'Educação'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,alunos,ser
2,escolas,qualidade
3,social,onde
4,mudanças,todo
5,escola,ter
6,discurso,social
7,comissão,sobre
8,necessidade,segundo
9,jovens,pessoas
10,pasta,parece


##### Top-10 palavras associadas aos Top-5 Documentos de 'Ministério'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,federal,desde
2,ministro,ter
3,público,social
4,militares,ser
5,valores,semana
6,governo,segundo
7,disse,país
8,rio,pasta
9,justiça,parte
10,chefe,médio


##### Top-10 palavras associadas aos Top-5 Documentos de 'Brasil'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,vida
2,sobre,vai
3,ainda,sobre
4,governo,país
5,ser,parte
6,presidente,mundo
7,anos,ministro
8,país,maior
9,sociais,governo
10,outros,fazer


##### Top-10 palavras associadas aos Top-5 Documentos de 'Economia'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,reforma,sobre
2,previdência,ser
3,mercado,relação
4,econ,pouco
5,governo,mercado
6,menos,dinheiro
7,mica,crise
8,política,agora
9,presidente,vez
10,ministro,tudo


#### Top-10 Documentos

Expansão de consulta para os top-10 documentos das consultas selecionadas inicialmente.

In [0]:
k = 10
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [0]:
table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

##### Top-10 palavras associadas aos Top-10 Documentos de 'Bolsonaro'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,presidente,presidente
2,brasil,fazer
3,governo,sobre
4,brasileiro,ser
5,ministro,dois
6,político,país
7,brasileira,outros
8,ainda,maior
9,paulo,disse
10,federal,dia


##### Top-10 palavras associadas aos Top-10 Documentos de 'Educação'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,alunos,sobre
2,escolas,ser
3,social,presidente
4,mudanças,onde
5,escola,ainda
6,discurso,ter
7,comissão,segundo
8,necessidade,pessoas
9,jovens,outras
10,pasta,maior


##### Top-10 palavras associadas aos Top-10 Documentos de 'Ministério'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,federal,ser
2,ministro,governo
3,público,ter
4,militares,país
5,valores,parte
6,governo,federal
7,disse,anos
8,rio,ainda
9,justiça,três
10,chefe,social


##### Top-10 palavras associadas aos Top-10 Documentos de 'Brasil'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,país
2,sobre,parte
3,ainda,governo
4,governo,vida
5,ser,sobre
6,presidente,ser
7,anos,brasileiro
8,país,brasileira
9,sociais,anos
10,outros,ainda


##### Top-10 palavras associadas aos Top-10 Documentos de 'Economia'

In [0]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,reforma,sobre
2,previdência,ser
3,mercado,econ
4,econ,todos
5,governo,ter
6,menos,outros
7,mica,mica
8,política,ainda
9,presidente,relação
10,ministro,pouco
